# From FASTQ to **Personalized FASTA (SNP Excerpts)** for Genome Language Models  
**fastp ➜ Parabricks FQ2BAM (GPU WGS) ➜ Parabricks HaplotypeCaller (GPU) ➜ bcftools filter ➜ SNP-window consensus FASTA**

---

Genome language models (GLMs) are most informative when they “see” **subject-specific variants** rather than a generic reference. This notebook shows a reproducible, GPU-accelerated path from raw short-read FASTQs to a **personalized FASTA composed of SNP-centered sequence windows**, ready for downstream LM work. Along the way we keep to community best practices while noting where GLM-specific choices (tokenization, windowing, heterozygosity) matter.

### A (very) brief lineage

- **FASTQ preprocessing got faster & simpler.** Instead of chaining multiple tools for QC, adapter/quality trimming, and UMI handling, **fastp** consolidated these into one multithreaded binary with built-in reports. [fastp paper][fastp-paper]; [fastp GitHub][fastp-github].  
- **Mapping standardized on BWA-MEM + SAM/BAM.** BWA-MEM (2013) became the workhorse for short/long reads; SAM/BAM (2009) provided the interoperable alignment container used everywhere. [BWA-MEM][bwa-mem]; [SAM/BAM][sam-bam].  
- **Variant calling best practices.** The **GATK** pipeline systematized mark-duplicates, BQSR, and HaplotypeCaller’s **local de-novo assembly** for robust germline calling. [GATK Best Practices][gatk-best-practices].  
- **GPUs made production-scale genomics practical.** Suites like **NVIDIA Parabricks** port the canonical CPU tools to GPUs (same results, much faster), enabling cohort-scale turnarounds and reduced cloud cost. [Parabricks FQ2BAM][pbr-fq2bam]; [Parabricks overview][pbr-overview]; [GPU scaling][gpu-scaling].

---

## 1) Preprocess reads with **fastp**

**Why:** Single-step adapter/quality trimming with rich QC, at speed.  
**Example:**
```bash
fastp -i sample_R1.fastq.gz -I sample_R2.fastq.gz \
      -o sample.trim.R1.fq.gz -O sample.trim.R2.fq.gz \
      --detect_adapter_for_pe --cut_mean_quality 20 \
      --length_required 50 --thread 16 \
      --html fastp.html --json fastp.json
````

---

## 2) Map & produce BAM with **Parabricks FQ2BAM** (GPU)

**Why:** GPU-accelerated BWA-MEM + sorting + duplicate marking (+ optional BQSR) with CPU-parity outputs.

```bash
pbrun fq2bam \
  --ref hg38.fa \
  --in-fq sample.trim.R1.fq.gz sample.trim.R2.fq.gz \
  --out-bam sample.bam \
  --knownSites dbsnp.vcf.gz \
  --markdups --BQSR --num-threads 32
```

References: [BWA-MEM][bwa-mem], [SAM/BAM][sam-bam], [GATK Best Practices][gatk-best-practices], [Parabricks FQ2BAM][pbr-fq2bam], [Parabricks overview][pbr-overview], [GPU scaling][gpu-scaling].

---

## 3) Call germline variants with **Parabricks HaplotypeCaller** (GPU)

**Why:** GATK’s **local assembly** improves accuracy in complex loci; Parabricks mirrors behavior at GPU speeds.

```bash
pbrun haplotypecaller \
  --ref hg38.fa \
  --in-bam sample.bam \
  --out-variants sample.g.vcf.gz \
  --emit-ref-confidence GVCF --gvcf
```

(Optionally joint-genotype multiple gVCFs before filtering.)

References: [HaplotypeCaller docs][haplotypecaller-docs], [GATK Best Practices][gatk-best-practices], [Parabricks overview][pbr-overview].

---

## 4) Filter to **high-confidence SNPs**

Use transparent, auditable filters (tune to coverage/organism). Example:

```bash
# Keep biallelic SNPs with minimal depth/quality standards
bcftools view -m2 -M2 -v snps sample.g.vcf.gz -Oz -o sample.snps.vcf.gz
tabix -p vcf sample.snps.vcf.gz

bcftools filter -i 'TYPE="snp" && DP>=10 && GQ>=20 && MQ>=40' \
  sample.snps.vcf.gz -Oz -o sample.snps.filtered.vcf.gz
tabix -p vcf sample.snps.filtered.vcf.gz
```

References: [bcftools manual][bcftools-manual].

---

## 5) Build **SNP-centered FASTA windows** (personalized)

For language models that operate on **short excerpts**, extract windows ±*W* bp around each SNP and write a FASTA with one record per window.

**5a. Prepare BED of SNP loci and expand to windows**

```bash
# 1-bp BED of SNP sites (BED is 0-based, half-open)
bcftools query -f '%CHROM\t%POS\t%ID\n' sample.snps.filtered.vcf.gz \
| awk 'BEGIN{OFS="\t"} {start=$2-1; print $1, start, $2, $3}' \
> snps.1bp.bed

# Expand by W bp on each side (example: W=100, giving 201-bp windows)
bedtools slop -i snps.1bp.bed -g hg38.chrom.sizes -b 100 > snps.200bp.bed
```

**5b. Write consensus sequence per window (choose het handling)**

```bash
# Haplotype options:
#   -H 1 or -H 2    -> pick a phased haplotype (good for generation)
#   -I (--iupac-codes) -> retain heterozygosity via IUPAC (good for embeddings)

bcftools consensus \
  -f hg38.fa \
  -s SAMPLE_ID \
  -H 1 \
  -R snps.200bp.bed \
  --allow-overlaps \
  sample.snps.filtered.vcf.gz > SAMPLE.snp_windows.fa
```

Notes: `bcftools consensus` applies genotype calls onto the reference and can limit output to target regions (`-r/-R`). IUPAC output preserves heterozygous ambiguity; haplotype selection produces unambiguous A/C/G/T strings. [bcftools consensus how-to][bcftools-consensus]; [bcftools manual][bcftools-manual]. For region math, we used **BEDTools**. [BEDTools paper][bedtools-paper].

---

## Designing FASTA **for LMs**: NLU vs NLG

Modern genomics LMs vary in tokenization (single nucleotides, **k-mers**, or **BPE-style** units) and context length, which changes how you should package sequences. DNABERT popularized overlapping **k-mers** (often 6-mers) for classification tasks; newer models explore BPE-like tokenization and very long contexts (e.g., **HyenaDNA**). [DNABERT][dnabert-paper]; [DNABERT repo][dnabert-code]; [Nucleotide Transformer][nt-paper]; [DNABERT-2][dnabert2-paper]; [HyenaDNA][hyenadna-paper].

**If you’ll use *final-layer hidden states* (NLU/embedding features):**

* Prefer **fixed-length windows** (e.g., 201/401 bp) for consistent batching and pooling.
* **Keep heterozygosity** via **IUPAC** (`bcftools consensus -I`) so embeddings encode zygosity without duplicating windows.
* Use a clear FASTA header schema to carry metadata you’ll need later (e.g., `>rs123|chr1:1,234,567|W=200|GENE=...`).
* Downstream, pool token embeddings or use a special aggregate token (analogous to **\[CLS]** in BERT) for per-window features. [BERT][bert-paper].

**If you’ll use *raw token logits* (NLG/generative scoring or sampling):**

* Prefer **unambiguous haplotype strings** (`-H 1` or `-H 2`), avoiding IUPAC—ambiguous codes can confuse next-token prediction.
* Maintain **contiguity** within each record (do not interleave far-apart windows in one sequence); if you concatenate windows, separate with a consistent **sentinel motif** (e.g., long runs of `N`) your tokenizer handles gracefully.
* Match the model’s tokenizer: for **k-mer LMs**, ensure record lengths align to the stride; for **BPE/single-nt LMs**, plain A/C/G/T/N is fine. [DNABERT][dnabert-paper]; [DNABERT-2][dnabert2-paper].

> **Rule of thumb:**
>
> * **NLU/embeddings:** IUPAC hets + fixed windows → feature vectors for classifiers/regressors.
> * **NLG/logits:** pick a haplotype, avoid ambiguity, keep sequences contiguous for stable next-token behavior.

---

## Why SNP-window FASTA helps

SNP-centric windows give LMs **direct access to the causal bases** and their immediate context, reducing sequence redundancy and I/O while retaining the signal most relevant to variant-aware tasks. This format also scales well to cohorts using **GPU pipelines** for mapping/calling and efficient **bcftools** filtering/consensus for final assembly. [GPU scaling][gpu-scaling]; [bcftools manual][bcftools-manual]; [bcftools consensus][bcftools-consensus].

---

## Minimal end-to-end checklist

1. **fastp**: trim & QC → `sample.trim.*.fq.gz`.
2. **Parabricks FQ2BAM**: map, sort, mark-dups (±BQSR) → `sample.bam`.
3. **Parabricks HaplotypeCaller**: call variants → `sample.g.vcf.gz` (±joint-genotype).
4. **bcftools**: select high-confidence **SNPs** → `sample.snps.filtered.vcf.gz`.
5. **BEDTools**: expand SNPs to ±*W* windows → `snps.<2W+1>bp.bed`.
6. **bcftools consensus**: write **personalized SNP-window FASTA** → `SAMPLE.snp_windows.fa`.

---

## References (selected)

* **Preprocessing & Alignment**
  Chen *et al.* 2018 — **fastp**: ultra-fast FASTQ preprocessor. [fastp paper][fastp-paper]; [fastp GitHub][fastp-github].
  Li 2013 — **BWA-MEM**. [BWA-MEM][bwa-mem].
  Li *et al.* 2009 — **SAM/BAM** format. [SAM/BAM][sam-bam].

* **Variant Calling & GPUs**
  Van der Auwera *et al.* 2013 — **GATK Best Practices**. [GATK Best Practices][gatk-best-practices].
  **Parabricks** docs — **FQ2BAM**, **HaplotypeCaller**. [pbr-fq2bam][pbr-fq2bam]; [haplotypecaller-docs][haplotypecaller-docs]; [Parabricks overview][pbr-overview].
  Taylor-Weiner *et al.* 2019 — **GPU scaling** in genomics. [GPU scaling][gpu-scaling].

* **Consensus & Regions**
  **bcftools** manual & how-to — filtering and `consensus`. [bcftools-manual][bcftools-manual]; [bcftools-consensus][bcftools-consensus].
  Quinlan & Hall 2010 — **BEDTools**. [BEDTools paper][bedtools-paper].

* **Genome LMs & tokenization**
  Ji *et al.* 2021 — **DNABERT**. [DNABERT][dnabert-paper]; [DNABERT repo][dnabert-code].
  Dalla-Torre *et al.* 2023/2025 — **Nucleotide Transformer**. [NT paper][nt-paper].
  Zhou *et al.* 2024 — **DNABERT-2** (BPE tokenization). [DNABERT-2][dnabert2-paper].
  Nguyen *et al.* 2023 — **HyenaDNA** (ultra-long contexts). [HyenaDNA][hyenadna-paper].
  Devlin *et al.* 2018/2019 — **BERT** (NLU/\[CLS] pooling). [BERT][bert-paper].

---

<!-- Reference-style link definitions -->

[fastp-paper]: https://academic.oup.com/bioinformatics/article/34/17/i884/5093234 "fastp: an ultra-fast all-in-one FASTQ preprocessor"
[fastp-github]: https://github.com/OpenGene/fastp "OpenGene/fastp"
[bwa-mem]: https://arxiv.org/pdf/1303.3997 "Aligning sequence reads, clone sequences and assembly contigs with BWA-MEM"
[sam-bam]: https://pmc.ncbi.nlm.nih.gov/articles/PMC2723002/ "The Sequence Alignment/Map format and SAMtools"
[gatk-best-practices]: https://pmc.ncbi.nlm.nih.gov/articles/PMC4243306/ "From FastQ data to high confidence variant calls (GATK Best Practices)"
[pbr-fq2bam]: https://docs.nvidia.com/clara/parabricks/4.4.0/Documentation/ToolDocs/man_fq2bam.html "Parabricks FQ2BAM (BWA-MEM + GATK)"
[haplotypecaller-docs]: https://docs.nvidia.com/clara/parabricks/4.4.0/Documentation/ToolDocs/man_haplotypecaller.html "Parabricks HaplotypeCaller"
[pbr-overview]: https://docs.nvidia.com/clara/parabricks/latest/Tutorials/FQ2BAM_Tutorial.html "NVIDIA Parabricks overview & FQ2BAM tutorial"
[gpu-scaling]: https://genomebiology.biomedcentral.com/articles/10.1186/s13059-019-1836-7 "Scaling computational genomics to millions of individuals with GPUs"
[bcftools-manual]: https://samtools.github.io/bcftools/bcftools.html "bcftools manual"
[bcftools-consensus]: https://samtools.github.io/bcftools/howtos/consensus-sequence.html "bcftools consensus how-to"
[bedtools-paper]: https://academic.oup.com/bioinformatics/article/26/6/841/244688 "BEDTools: a flexible suite of utilities for comparing genomic features"
[dnabert-paper]: https://academic.oup.com/bioinformatics/article/37/15/2112/6128680 "DNABERT: pre-trained BERT for DNA sequences"
[dnabert-code]: https://github.com/jerryji1993/DNABERT "DNABERT GitHub repository"
[nt-paper]: https://www.biorxiv.org/content/10.1101/2023.01.11.523679v3 "The Nucleotide Transformer"
[dnabert2-paper]: https://arxiv.org/html/2306.15006v2 "DNABERT-2: Efficient Foundation Model for Genomics"
[hyenadna-paper]: https://papers.neurips.cc/paper_files/paper/2023/file/86ab6927ee4ae9bde4247793c46797c7-Paper-Conference.pdf "HyenaDNA: Long-Range Genomic Sequence Modeling"
[bert-paper]: https://arxiv.org/abs/1810.04805 "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding"

```
::contentReference[oaicite:0]{index=0}
```


## Import wrapper functions for pipeline

In [ ]:
import os
import subprocess
from typing import Dict

# Configurations
THREADS = 16
MEMORY = 128
MIN_VAR_QUAL = 30
MIN_DEPTH = 6
MIN_VAR_FREQ = 0.6  # For haploid, variants should be majority frequency or greater
MAX_DEPTH = 500


def trim_fastq(
    job: Dict[str, str],
    min_illumina_quality: int = 20,
    min_illumina_length: int = 50,
    min_illumina_complexity: int = 30,
):
    print(job["OUTPUT_PREFIX"])
    cmd = [
        "/opt/fastp",
        "-i",
        job["UNPROCESSED_FASTQ_R1"],
        "-I",
        job["UNPROCESSED_FASTQ_R2"],
        "-o",
        job["FASTQ_R1"],
        "-O",
        job["FASTQ_R2"],
        "-w",
        "16",
        "-q",
        str(min_illumina_quality),
        "--detect_adapter_for_pe",
        "--length_required",
        str(min_illumina_length),
        "-c",
        "-y",
        "-Y",
        str(min_illumina_complexity),
    ]
    with open(job["LOG_FILE"], "w") as f_out:
        subprocess.run(
            cmd,
            stdout=f_out,
            stderr=subprocess.STDOUT,
            text=True,
            check=True,
        )


def alignment(job: Dict[str, str]):
    print(job["OUTPUT_PREFIX"])
    cmd = [
        "pbrun",
        "fq2bam",
        "--ref",
        job["REF_FASTA"],
        "--in-fq",
        job["FASTQ_R1"],
        job["FASTQ_R2"],
        "--out-bam",
        job["BAM_FILE"],
    ]
    with open(job["LOG_FILE"], "a") as f_out:
        subprocess.run(
            cmd,
            stdout=f_out,
            stderr=subprocess.STDOUT,
            text=True,
            check=True,
        )


def variant_calling(job: Dict[str, str]):
    print(job["OUTPUT_PREFIX"])
    cmd = [
        "pbrun",
        "haplotypecaller",
        "--ref",
        job["REF_FASTA"],
        "--in-bam",
        job["BAM_FILE"],
        "--out-variants",
        job["RAW_VCF"],
        "--ploidy",
        "1",
        "--minimum-mapping-quality",
        "20",
        "--haplotypecaller-options",
        "-standard-min-confidence-threshold-for-calling 10",
    ]
    with open(job["LOG_FILE"], "a") as f_out:
        subprocess.run(
            cmd,
            stdout=f_out,
            stderr=subprocess.STDOUT,
            text=True,
            check=True,
        )


def basic_snp_filter(job: Dict[str, str]):
    print(job["OUTPUT_PREFIX"])
    cmd = [
        "bcftools",
        "view",
        "-i",
        f"QUAL >= {MIN_VAR_QUAL} && INFO/DP >= {MIN_DEPTH} && INFO/DP <= {MAX_DEPTH}",
        job["RAW_VCF"],
        "-O",
        "v",
        "-o",
        job["FILTERED_VCF"],
    ]
    with open(job["LOG_FILE"], "a") as f_out:
        subprocess.run(
            cmd,
            stdout=f_out,
            stderr=subprocess.STDOUT,
            text=True,
            check=True,
        )


def haploid_snp_filter(job: Dict[str, str]):
    print(job["OUTPUT_PREFIX"])
    cmd = [
        "bcftools",
        "view",
        "-i",
        f"FORMAT/AD[0:1]/(FORMAT/AD[0:0]+FORMAT/AD[0:1]) >= {MIN_VAR_FREQ}",
        job["FILTERED_VCF"],
        "-O",
        "v",
        "-o",
        job["HIGH_CONF_VCF"],
    ]
    with open(job["LOG_FILE"], "a") as f_out:
        subprocess.run(
            cmd,
            stdout=f_out,
            stderr=subprocess.STDOUT,
            text=True,
            check=True,
        )


def generate_fasta(job: Dict[str, str]):
    print(job["OUTPUT_PREFIX"])
    cmd1 = [
        "bcftools",
        "norm",
        "-f",
        job["REF_FASTA"],
        "-m",
        "-both",
        job["HIGH_CONF_VCF"],
        "-O",
        "v",
        "-o",
        f"{job['HIGH_CONF_VCF']}.normalized.vcf",
    ]
    cmd2 = [
        "bcftools",
        "view",
        f"{job['HIGH_CONF_VCF']}.normalized.vcf",
        "-Oz",
        "-o",
        f"{job['HIGH_CONF_VCF']}.normalized.vcf.gz",
    ]
    cmd3 = ["bcftools", "index", f"{job['HIGH_CONF_VCF']}.normalized.vcf.gz"]
    cmd4 = [
        "bcftools",
        "consensus",
        "-f",
        job["REF_FASTA"],
        "-o",
        job["CONSENSUS_FASTA"],
        f"{job['HIGH_CONF_VCF']}.normalized.vcf.gz",
    ]
    with open(job["LOG_FILE"], "a") as f_out:
        for cmd in [cmd1, cmd2, cmd3, cmd4]:
            subprocess.run(
                cmd,
                stdout=f_out,
                stderr=subprocess.STDOUT,
                text=True,
                check=True,
            )

### Preprocess datasets

In [ ]:
base = "/workspace/datasets"
study_dir = f"{base}/haploid"
bam_dir = f"{study_dir}/bam"
vcf_dir = f"{study_dir}/vcf"
tmp_dir = f"{study_dir}/tmp"
fastq_dir = f"{study_dir}/fastq"
fastq_raw_delimiters = ["_unprocessed_illumina_"]

# Create directories for files
!mkdir -p $base
!mkdir -p $study_dir
!mkdir -p $bam_dir
!mkdir -p $vcf_dir
!mkdir -p $tmp_dir

In [ ]:
# Preprocess fasta file
fasta_file = f"{study_dir}/fasta/GCA_000027005.1_ASM2700v1_genomic.fna"
genomic_gtf = f"{study_dir}/fasta/genomic.gtf"

# Index fasta with BWA
!/opt/bwa/bwa index $fasta_file

### Build jobs

In [ ]:
fastq_files = [
    f"{fastq_dir}/{i}"
    for i in os.listdir(fastq_dir)
    if all([j in i for j in fastq_raw_delimiters])
]
sample_names = ["your-sample-names"]
# My process to identify filenames based on path
# sample_names = sorted(list(set([i.split("fastq_unprocessed_illumina_")[-1].split("_R")[0] for i in fastq_files])))

jobs = [
    {
        "UNPROCESSED_FASTQ_R1": [
            i for i in fastq_files if sample_name in i and "_R1." in i
        ][0],
        "UNPROCESSED_FASTQ_R2": [
            i for i in fastq_files if sample_name in i and "_R2." in i
        ][0],
        "FASTQ_R1": [
            i.replace("_unprocessed_", "_trimmed_")
            for i in fastq_files
            if sample_name in i and "_R1." in i
        ][0],
        "FASTQ_R2": [
            i.replace("_unprocessed_", "_trimmed_")
            for i in fastq_files
            if sample_name in i and "_R2." in i
        ][0],
        "REF_FASTA": fasta_file,
        "ANNOTATION_GTF": genomic_gtf,
        "OUTPUT_PREFIX": sample_name,
        "BAM_FILE": f"{study_dir}/bam/{sample_name}.sorted.bam",
        "RAW_VCF": f"{study_dir}/vcf/{sample_name}.raw.vcf",
        "FILTERED_VCF": f"{study_dir}/vcf/{sample_name}.filtered.vcf",
        "HIGH_CONF_VCF": f"{study_dir}/vcf/{sample_name}.high_confidence.vcf",
        "CONSENSUS_FASTA": f"{study_dir}/fasta/{sample_name}.consensus.fasta",
        "STATS_FILE": f"{study_dir}/{sample_name}.stats.txt",
        "LOG_FILE": f"{study_dir}/{sample_name}.pipeline.log",
    }
    for sample_name in sample_names
]
print(len(jobs))
jobs

### Run pipeline

In [ ]:
for job in jobs:
    trim_fastq(job)
    alignment(job)
    variant_calling(job)
    basic_snp_filter(job)
    haploid_snp_filter(job)
    generate_fasta(job)

## References

1. Chen S. *et al.* (2018) **fastp**: an ultra-fast all-in-one FASTQ preprocessor. *Bioinformatics*. ([paper][fastp-paper], [code][fastp-github])
2. Li H. (2013) **BWA-MEM**: aligning sequence reads and contigs. *arXiv*. ([preprint][bwa-mem])
3. Li H. *et al.* (2009) **The SAM/BAM format and SAMtools**. *Bioinformatics*. ([article][sam-bam])
4. Van der Auwera G.A. *et al.* (2013) **From FastQ data to high-confidence variant calls**. *Curr Protoc Bioinformatics*. ([overview][gatk-best-practices])
5. **GATK HaplotypeCaller** documentation. ([page][haplotypecaller-docs])
6. **NVIDIA Parabricks**: FQ2BAM tool and suite overview. ([FQ2BAM][pbr-fq2bam], [overview][pbr-overview])
7. Taylor-Weiner A. *et al.* (2019) **Scaling computational genomics with GPUs**. *Genome Biology*. ([article][gpu-scaling])
8. **bcftools** manual and consensus how-to. ([manual][bcftools-manual], [how-to][bcftools-consensus])